In [4]:
# データの読み込み(sklearn)
import torch
from skorch import NeuralNetClassifier
from torch import nn
import torch.nn.functional as F
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
mnist = fetch_mldata('MNIST original', data_home='./data/mnist')
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')
X /= 255
XCnn = X.reshape(-1, 1, 28, 28)
XCnn_train, XCnn_test, y_train, y_test = train_test_split(XCnn, y, test_size=0.25, random_state=42)
# Networkの設計(PyTorch)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1600, 128) # 1600 = number channels * width * height
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, x.size(1) * x.size(2) * x.size(3)) # flatten over channel, height and width = 1600
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.softmax(x, dim=-1)
        return x
# ラッパーを使う(skorch)
net = NeuralNetClassifier(
    Net,
    max_epochs=10,
    lr=1,
    optimizer=torch.optim.Adadelta,
    # use_cuda=True,  # uncomment this to train with CUDA
)
# training
net.fit(XCnn_train, y_train)

# test
y_pred = net.predict(XCnn_test)
print(classification_report(y_test, y_pred))

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        0.4107       0.9741        0.0838  50.1330
      2        0.1332       0.9812        0.0603  48.3953
      3        0.1077       0.9845        0.0467  59.6833
      4        0.0896       0.9863        0.0420  50.3603
      5        0.0808       0.9863        0.0394  47.4268
      6        0.0733       0.9882        0.0358  49.1700
      7        0.0672       0.9890        0.0347  52.3128
      8        0.0636       0.9889        0.0342  48.8148
      9        0.0590       0.9887        0.0339  46.8803
     10        0.0548       0.9901        0.0306  47.4012
             precision    recall  f1-score   support

          0       1.00      0.99      1.00      1677
          1       0.99      1.00      1.00      1935
          2       0.98      0.99      0.99      1767
          3       0.99      0.99      0.99      1766
          4       0.98      0.99      

In [ ]:
print()